In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [5]:
KEY=os.getenv("mykey")

In [6]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

C:\Users\PUJA KUMARI\AppData\Local\Temp\ipykernel_17036\3870565440.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)


In [18]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [19]:
#FORMAT IN WHICH WE WANT OUR MCQ
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [20]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [21]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [22]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\PUJA KUMARI\AppData\Local\Temp\ipykernel_7440\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [23]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [24]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE)

In [25]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [26]:
#sequentialchai is a class and genetrate_evaluate_chain is an object
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                            input_variables=["text", "number", "subject", "tone", "response_json"],
                            output_variables=["quiz", "review"], verbose=True
)

In [27]:
file_path=r"C:\Users\PUJA KUMARI\Desktop\mcqgen\data.txt"

In [28]:
file_path

'C:\\Users\\PUJA KUMARI\\Desktop\\mcqgen\\data.txt'

In [29]:
#read the data file
with open(file_path,'r') as file:
    TEXT=file.read()

In [30]:
#inside text all data is present
print(TEXT)

The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[21] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[22] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[23][24] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automaton created in the early 1800s.[25]

Artificial Intelligence research began in the 1950s with works like Compu

In [31]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [32]:
NUMBER=5 #no of question we want to genrate
SUBJECT="generative ai"
TONE="simple"

In [33]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    #calling thus generate_evaluate_chain here
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )


C:\Users\PUJA KUMARI\AppData\Local\Temp\ipykernel_7440\3379600259.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[21] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[22] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[23][24] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automato

In [34]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3290
Prompt Tokens:2454
Completion Tokens:836
Total Cost:0.005353


In [35]:
response

{'text': 'The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[21] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[22] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[23][24] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet\'s automaton created in the early 1800s.[25]\n\nArtificial Intelligence research began in the 1950s with wor

In [36]:
quiz=response.get("quiz")

In [37]:

quiz=json.loads(quiz)

In [38]:
#create dataframe in list format
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [39]:
quiz_table_data

[{'MCQ': 'What year was the academic discipline of artificial intelligence established at a research workshop held at Dartmouth College?',
  'Choices': 'a: 1950 | b: 1956 | c: 1960 | d: 1970',
  'Correct': 'b'},
 {'MCQ': 'Which ancient Greek inventors were described as having designed machines capable of writing text, generating sounds, and playing music?',
  'Choices': 'a: Daedalus and Hero of Alexandria | b: Archimedes and Pythagoras | c: Plato and Aristotle | d: Socrates and Homer',
  'Correct': 'a'},
 {'MCQ': 'Who created the computer program AARON to generate paintings in the early 1970s?',
  'Choices': 'a: Harold Cohen | b: Andrey Markov | c: Hero of Alexandria | d: Daedalus',
  'Correct': 'a'},
 {'MCQ': 'What technology was used in the 1980s and 1990s to generate sequences of actions to reach a specified goal?',
  'Choices': 'a: Deep Learning | b: Markov Chains | c: Generative AI Planning | d: Neural Networks',
  'Correct': 'c'},
 {'MCQ': 'Which country is leading the world in a

In [40]:
#convert to dataframe
#
quiz=pd.DataFrame(quiz_table_data)

In [41]:
#store this in csv file

quiz.to_csv("generativeai.csv",index=False)

In [42]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'10_03_2024_19_52_24'